# 梯度提升回归树 (GBRT) 早停与调优

## 算法原理

GBRT (Gradient Boosted Regression Trees) 是梯度提升用于回归任务的实现。本节重点介绍：

1. **早停策略**: 防止过拟合，找到最优迭代次数
2. **staged_predict**: 获取每轮迭代的预测结果
3. **warm_start**: 增量训练，动态调整树的数量

## 早停的重要性

- 梯度提升容易过拟合
- 太多的树会导致验证误差上升
- 早停可以自动找到最佳迭代次数

In [ ]:
# 导入必要的库
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 设置随机种子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 1. 数据准备

In [ ]:
# 加载数据
iris = load_iris()
X, y = iris.data, iris.target

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print(f"训练集大小: {X_train.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")

## 2. 使用 staged_predict 寻找最佳迭代次数

`staged_predict` 方法可以获取每轮迭代后的预测，用于分析学习曲线。

In [ ]:
# 训练一个有较多树的模型
gbrt = GradientBoostingRegressor(
    max_depth=2,
    n_estimators=120,
    learning_rate=0.1,
    random_state=RANDOM_STATE
)
gbrt.fit(X_train, y_train)

# 计算每个阶段的测试误差
errors = [mean_squared_error(y_test, y_pred) 
          for y_pred in gbrt.staged_predict(X_test)]

# 找到最佳迭代次数（误差最小的点）
best_n_estimators = np.argmin(errors) + 1
min_error = min(errors)

print(f"最佳迭代次数: {best_n_estimators}")
print(f"最小测试MSE: {min_error:.6f}")

# 使用最佳参数重新训练
gbrt_best = GradientBoostingRegressor(
    max_depth=2,
    n_estimators=best_n_estimators,
    learning_rate=0.1,
    random_state=RANDOM_STATE
)
gbrt_best.fit(X_train, y_train)

print(f"最佳模型 R²: {gbrt_best.score(X_test, y_test):.4f}")

## 3. 使用 warm_start 实现早停

`warm_start=True` 允许增量训练，可以在验证误差开始上升时停止。

In [ ]:
# 使用 warm_start 实现早停
gbrt_warm = GradientBoostingRegressor(
    max_depth=2,
    warm_start=True,           # 启用增量训练
    random_state=RANDOM_STATE
)

min_val_error = float("inf")
error_going_up = 0
best_iteration = 0

# 逐步增加树的数量
for n_estimators in range(1, 121):
    gbrt_warm.n_estimators = n_estimators
    gbrt_warm.fit(X_train, y_train)
    y_pred = gbrt_warm.predict(X_test)
    val_error = mean_squared_error(y_test, y_pred)
    
    if val_error < min_val_error:
        min_val_error = val_error
        best_iteration = n_estimators
        error_going_up = 0
    else:
        error_going_up += 1
        # 如果连续5轮误差没有改善，停止训练
        if error_going_up == 5:
            break

print(f"早停迭代次数: {gbrt_warm.n_estimators}")
print(f"最佳迭代次数: {best_iteration}")
print(f"最小验证MSE: {min_val_error:.6f}")
print(f"最终模型 R²: {gbrt_warm.score(X_test, y_test):.4f}")

## 4. 不同策略的对比

In [ ]:
# 对比不同配置的性能
configs = [
    ('无早停 (120棵树)', 120),
    ('staged_predict 最佳', best_n_estimators),
    ('warm_start 早停', gbrt_warm.n_estimators)
]

print("不同策略性能对比:")
print(f"{'策略':<25} {'树数量':>10} {'R²':>10}")
print("-" * 47)

for name, n_trees in configs:
    model = GradientBoostingRegressor(
        max_depth=2,
        n_estimators=n_trees,
        learning_rate=0.1,
        random_state=RANDOM_STATE
    )
    model.fit(X_train, y_train)
    r2 = model.score(X_test, y_test)
    print(f"{name:<25} {n_trees:>10} {r2:>10.4f}")

## 5. 单元测试验证

In [ ]:
def test_gbrt():
    """GBRT 功能测试"""
    
    # 测试1: staged_predict 应该返回正确数量的预测
    assert len(errors) == 120, "staged_predict 返回数量不正确"
    
    # 测试2: 最佳迭代次数应该在合理范围
    assert 1 <= best_n_estimators <= 120, "最佳迭代次数不在预期范围"
    
    # 测试3: 最佳模型应该有良好性能
    assert gbrt_best.score(X_test, y_test) >= 0.9, "最佳模型 R² 过低"
    
    # 测试4: warm_start 早停应该在合理位置停止
    assert gbrt_warm.n_estimators < 120, "早停未生效"
    
    # 测试5: 早停不应该显著损害性能
    assert gbrt_warm.score(X_test, y_test) >= 0.9, "早停后 R² 过低"
    
    print("所有测试通过!")

test_gbrt()

## 总结

### 早停策略

1. **staged_predict**: 训练完成后分析，找到最佳迭代点
2. **warm_start**: 边训练边监控，达到早停条件即停止
3. **early_stopping (sklearn 0.24+)**: 内置早停参数

### 最佳实践

1. 始终使用验证集监控过拟合
2. 设置合理的耐心参数（如连续5轮无改善）
3. 使用较小的学习率配合更多的树
4. 早停后可以用最佳迭代次数重新训练